In [ ]:
import os
import csv
import pandas as pd
import numpy as np

import datetime
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import squarify

from sklearn import model_selection, preprocessing, metrics
plt.style.use('fivethirtyeight')

print(os.getcwd())
print(os.listdir("../"))

In [ ]:
def clean_data(df):
    #since first column contains row number
#     df.drop(["Row_num"], axis=1)
    #removing  and kesEntityId as simillar fields are already present
    df = df.drop(columns=["Row_num","kesEntityId"],axis=1)
    df["Popularity"] = df["Popularity"].apply(pd.to_numeric,downcast='float')
    df["Year"] = df["Year"].apply(pd.to_numeric,downcast='unsigned')
    df["Month"] = df["Month"].apply(pd.to_numeric,downcast='unsigned')
    df = df.dropna(subset=['Venue'])
    return df

In [ ]:
df_pub = pd.read_csv("../input/scopus-data/ProcessedScopusData.csv")
# If this doesn't work, please change it to the path where the file is locaed in your PC
# df_pub = pd.read_csv("Data/ProcessedScopusData.csv")

In [ ]:
df_pub.head()

In [ ]:
def append_pub_score(df):
    print("Unique Publications from Scopus:",len(df_pub.Title.unique()))
    # Analysing Common Publications
    vfn_list = df.Venue.unique().tolist()
    scopus_list = df_pub.Title.unique().tolist()
    count = 0
    for pub in scopus_list:
        if pub in vfn_list:
            count +=1
    print("Publications present in Dataset:",count)
    print("Shape Before",df.shape)
    df = df[df.Venue.isin(scopus_list)]
    print("Shape After",df.shape)
    #score appending
    score_dict = pd.Series(df_pub.SJR.values,index=df_pub.Title).to_dict()
    df["Publication_Rank"] = df.apply(lambda row: score_dict[row["Venue"]],axis = 1)
    return df

def extract_field(row):
    val = str(row["Domain"])
    index = val.rfind("FN")
    val = val[index+3:len(val)]
    val = val.strip(":}] '")
    return val

def get_num_authors(row):
    vals = row["Authors"].split("},")
    return len(vals)
def get_open_access(df):
    def get_label(val):
        if val == "YES":
            return 1
        else:
            return 0
    #score appending
    open_dict = pd.Series(df_pub.OpenAccess.values,index=df_pub.Title).to_dict()
    df["OpenAccess"] = df.apply(lambda row: get_label(open_dict[row["Venue"]]),axis = 1)
    return df

def get_num_domains(row):
    if not pd.isnull(row["Domain"]):
        json_obj = ast.literal_eval(row["Domain"])
        if json_obj:
            return len(json_obj)
    return 0

In [ ]:
label_dict = {"C":0,"J":1,"CJ":2,"O":3}
def find_conference_type(row):
    if pd.isnull(row["Conference"]) and pd.isnull(row["Journal"]):
        return label_dict["O"]
    elif pd.isnull(row["Conference"]):
        return label_dict["J"]
    elif pd.isnull(row["Journal"]):
        return label_dict["C"]
    else:
        return label_dict["CJ"]

In [ ]:
def tidy_split(df, column, sep='|', keep=False):
    indexes = list()
    id_values = list()
    name_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            id_values.append(presplit)
        for value in values:
            indexes.append(i)
            val = value
            index1 = val.find("AuId")
            index2 = val.find("AfN")
            index3 = val.find("'S'")
            if index3 < index1:
                val = val[index1+4:index2]
                val = val.strip("', :")
            else:
                val = val[index1+4:index3]
                val = val.strip("', :")                
            id_values.append(val)
            index1 = value.find("AuN")
            val = value[index1+3:]
            val = val.strip("': }]")
            name_values.append(val)
    new_df = df.iloc[indexes, :].copy()
    del df
    new_df[column+"_Id"] = id_values
    del new_df
    new_new_df = new_df.iloc[:, :].copy()
    new_new_df[column+"_Name"] = name_values
    return new_new_df

In [ ]:
def process_year(df):
    # Make more sensible column names
    rename_dict = {"AA": "Authors", 
                   "C": "Conference", 
                   "CC": "Citation_count",
                   "ECC": "Expected_count",
                   "D": "Date", 
                   "F": "Domain", 
                   "J": "Journal", 
                   "RId": "References", 
                   "Ti": "Title", 
                   "VFN": "Venue", 
                   "W": "Words", 
                   "Y": "Year", 
                   "logprob":"Popularity",
                   "Unnamed: 0": "Row_num"}
    df.rename(index=str, columns=rename_dict, inplace=True)
    df.drop_duplicates(subset=['Title'], inplace=True)
    df["Month"] = pd.to_datetime(df["Date"]).dt.month
    df = clean_data(df)
    df = append_pub_score(df)
    df["Publication Type"] = df.apply(lambda row:find_conference_type(row),axis=1)
    df["Number_Of_authors"] = df.apply(lambda row:get_num_authors(row), axis = 1)
    return df

In [ ]:
# df = pd.read_csv("../input/mag-2013/mag_2013.csv")

In [ ]:
# only for year 2013
# df = df[df['D'] != "-20.259"]

In [ ]:
# df = process_year(df)

In [ ]:
# df.to_csv("mag_2013_processed.csv")

In [ ]:
print(os.listdir("../input/"))

In [ ]:
columns = ['Unnamed: 0','Authors', 'Conference','Citation_count','Date','Expected_count','Domain','Id','Journal','References','Title','Venue','Words','Year','Popularity','Month','Publication_Rank','Publication Type','Number_Of_authors']

In [ ]:
df = pd.DataFrame(columns = columns)
# please give a path that works on your machine. Keep all the processed files in this path
path = "../input/processedmagdata"
for file in os.listdir(path):
#     print(file)
    df_temp = pd.read_csv(path+"/"+file)
    df = df.append(df_temp)

In [ ]:
# df = get_open_access(df)

In [ ]:
# df.isna().sum()

In [ ]:
# df.head()

In [ ]:
# No duplicate values :)
# df[df.duplicated()]

In [ ]:
#size of data
# df.shape

In [ ]:
# df["Publication Type"].value_counts()

In [ ]:
# Number of rows with 0 citation count and 0 References
# df[ (df['References'].isnull()) & (df['Citation_count']==0)].shape[0]

In [ ]:
#?
df = df.drop(['Unnamed: 0'],axis = 1)

In [ ]:
#?
df.dropna(subset=['Publication_Rank'], how='all', inplace = True)

In [ ]:
sns.set(style="darkgrid",font_scale=1.3)
ax = sns.countplot(x="Publication Type", data=df)

In [ ]:
# paper publication throughout the period
plt.figure(figsize=(13,6))
sns.set(style="darkgrid",font_scale=1.3)
ax = sns.countplot(x="Year", data=df)
plt.suptitle("Distribution of Paper published in Years")

In [ ]:
# paper publication around a year
plt.figure(figsize=(13,6))
sns.set(style="darkgrid",font_scale=1.3)
ax = sns.countplot(x="Month", data=df)
ax.set_xticklabels(["January","February","March", "April", "May", "June", "July", "August", "September", "October","November", "December"])
plt.suptitle("Number of paper published in each month")

In [ ]:
# max(df.Number_Of_authors)

In [ ]:
# df.sort_values(by=["Number_Of_authors"], ascending=False).head()

In [ ]:
# Number of authors for the paper, Group data according to number of authors
bins = [0, 10, 20, 40, 60, 80, 100, 200, 400, 500, np.inf]
# author_groups = pd.cut(df["Number_Of_authors"], bins)
# df["author_groups"] = author_groups
plt.figure(figsize=(10,4))
sns.set(style="darkgrid",font_scale=1.1)
sss = pd.cut(df['Number_Of_authors'], bins=bins)
ax = sss.value_counts(sort=False).plot.bar(rot=15)
for index, row in sss.iterrows():
    g.text(row.name,row.tip, row.total_bill,2, color='black', ha="center")
# sns.distplot(sss)

plt.suptitle("Number of authors for given paper")

In [ ]:
sss.value_counts()

In [ ]:
import json
import ast
# obj = json.loads("[{'AuId': 2168721218, 'S': 1, 'AuN': 'emelia j benjamin'},]")
obj = ast.literal_eval(df["Authors"][24093])
print(len(obj))
print(json.dumps(obj[0], indent=2))
universities = { _obj["AfN"] for _obj in obj if "AfN" in _obj}
print(universities)

In [ ]:
df["Topic"] = df.apply(lambda row: extract_field(row),axis = 1)

In [ ]:
# Keeping the top topics only
top_topics = ["medicine","biology","chemistry","physics","psychology","materials science","economics","mathematics","geology","computer science","sociology","environmental science","engineering","business","political science","geography","history","philosophy","artificial intelligence","art"]
topic_list = []
for index,row in df.iterrows():
    if row["Topic"] not in top_topics:
        topic_list.append(df_pub[df_pub["Title"] == row["Venue"]]["Subject Area"].tolist()[0])
    else:
        topic_list.append(row["Topic"])
df["Topic"] = topic_list
del topic_list

In [ ]:
plt.figure(figsize=(13,6))
top_10 = df["Topic"].value_counts()[0:10]
print(top_10)
top_10.plot(kind = "bar")

In [ ]:
topic_list = df["Topic"].tolist()
topic_dict = {}
i=0
for topic in topic_list:
    topic_dict[topic] = i
    i += 1

In [ ]:
venue_list = df["Venue"].tolist()
venue_dict = {}
i=0
for venue in venue_list:
    venue_dict[venue] = i
    i += 1

In [ ]:
# df["Topic_Label"] = df.apply(lambda row: topic_dict[row["Topic"]],axis = 1)
df["Year Since Publication"] = df.apply(lambda row: 2018-row["Year"],axis =1)
df["Venue_label"] = df.apply(lambda row: venue_dict[row["Venue"]],axis = 1)
df["Topic_label"] = df.apply(lambda row: topic_dict[row["Topic"]],axis = 1)

In [ ]:
del venue_list
del venue_dict
del topic_list
del topic_dict

In [ ]:
outlink_map = {}
inlink_map = {}
df_new = df[df["References"].notnull()]
for index, row in df_new.iterrows():
    ref = row["References"]
    ref_list = list(map(int,ref.strip("[]'").split(",")))
    outlink_map[row["Id"]] = ref_list
    for ref in ref_list:
        listi = inlink_map.get(ref, [])
        listi.append(row["Id"])
        inlink_map[ref] = listi

In [ ]:
def calculate_page_rank():
    count = 0
    page_year = {row["Id"]:row["Year"] for _t, row in df.iterrows()}
    year_citation_count = { page_year[index]:0 for index in page_year}
    year_paper_count = { page_year[index]:0 for index in page_year}
    avg_year_citation_count = {}
    for index in page_year:
        year = page_year[index]
        len_outlink = len(outlink_map.get(index, []))
        if len_outlink > 0:
            year_citation_count[year] += len_outlink
            year_paper_count[year] += 1
    for year in year_citation_count:
        if year_paper_count[year] > 0:
            avg_year_citation_count[year] = year_citation_count[year]/year_paper_count[year]
    page_rank = {}
    updated_page_rank = {}
    for index in df["Id"]:
        page_rank[index] = 1
    while True:
        count += 1
        flag = True
        for key in page_rank:
            cs = page_rank[key]
            if key in inlink_map:
                inlink_list = inlink_map[key]
                ns = 0
                for link in inlink_list:
                    if link in page_rank and link in outlink_map:
                        ns += page_rank[link]/len(outlink_map[link])
                #ns = 0.15 + (0.85 * ns)
                ns = 0.15 + float(0.85) * (ns/avg_year_citation_count[page_year[key]])
                if cs != ns:
                    flag = False
                updated_page_rank[key] = ns
        if flag == True:
            print(count)
            break
        page_rank = updated_page_rank
        updated_page_rank = {}
        max_score = max(page_rank.values())
        print(f"max score is {max_score}")
        page_rank = { index:score/max_score for index, score in page_rank.items()}
        return page_rank

In [ ]:
page_rank = calculate_page_rank()
print(max(page_rank.values()))
print(sum(page_rank.values()))

In [ ]:
del inlink_map
del outlink_map

In [ ]:
df["page_rank"] = 0
def update_rank(row):
    if row["Id"] in page_rank:
        return page_rank[row["Id"]]
    else:
        return 0
df["page_rank"] = df.apply(lambda row: update_rank(row),axis = 1)
df["page_rank"][0:10]

In [ ]:
# df.to_csv("ranked_processed_data.csv")

In [ ]:
del page_rank

In [ ]:
cf_list = {}
cf_score = {}
i = 0
for index,row in df.iterrows():
    info_list = cf_list.get(row["Venue"],[0,0])
    info_list[0] += row["page_rank"]
    info_list[1] += 1
    cf_list[row["Venue"]] = info_list
    i += 1
for key,value in cf_list.items():
    score = value[0]/value[1]
    cf_score[key] = score

In [ ]:
del cf_list

In [ ]:
df.head()

In [ ]:
# author_score = {}
# author_list = {}
# for index,row in df.iterrows():
#     values = row["Authors"].split("},")
#     for value in values:
#         val = value
#         index1 = val.find("AuId")
#         index2 = val.find("AfN")
#         index3 = val.find("'S'")
#         if index3 < index1:
#             val = val[index1+4:index2]
#             val = val.strip("', :")
#         else:
#             val = val[index1+4:index3]
#             val = val.strip("', :")                
#         auth_id = val
#         index1 = value.find("AuN")
#         val = value[index1+3:]
#         val = val.strip("': }]")
#         auth_name = val
#         auth_info = author_list.get(auth_id,["",0,0])
#         auth_info[0] = auth_name
#         auth_info[1] += row["page_rank"] * cf_score[row["Venue"]]
#         auth_info[2] += 1
#         author_list[auth_id] = auth_info
# for key, value in author_list.items():
#     auth_info = author_list.get(key,["",0,0])
#     author_score[key] = auth_info[1]/auth_info[2]

In [ ]:
author_score = {}
author_list = {}
for index,row in df.iterrows():
    values = row["Authors"].split("},")
    for value in values:
        val = value
        index1 = val.find("AuId")
        index2 = val.find("AfN")
        index3 = val.find("'S'")
        if index3 < index1:
            val = val[index1+4:index2]
            val = val.strip("', :")
        else:
            val = val[index1+4:index3]
            val = val.strip("', :")                
        auth_id = val
        index1 = value.find("AuN")
        val = value[index1+3:]
        val = val.strip("': }]")
        auth_name = val
        auth_info = author_list.get(auth_id,["",0,0,0,set()])
        auth_info[0] = auth_name
        auth_info[1] += row["page_rank"] * cf_score[row["Venue"]]
        auth_info[2] += 1
        auth_info[3] += row["Citation_count"]
        auth_info[4].add(row["Venue"])
        author_list[auth_id] = auth_info
for key, value in author_list.items():
    auth_info = author_list.get(key,["",0,0])
    author_score[key] = auth_info[1]/auth_info[2]

In [ ]:
def get_author_papers(auth_id):
    auth_info = author_list.get(auth_id,["",0,0,0])
    return auth_info[2]
def get_author_citation(auth_id):
    auth_info = author_list.get(auth_id,["",0,0,0])
    return auth_info[3]
def get_author_name(auth_id):
    auth_info = author_list.get(auth_id,["",0,0,0])
    return auth_info[0]
def get_author_publications(auth_id):
    auth_info = author_list.get(auth_id,["",0,0,0])
    return len(auth_info[4])

In [ ]:
columns = ["Author Id","Author Name","Number Of Papers","Number Of Publications","Author Score","Number Of Citations"]
df_authors = pd.DataFrame(np.nan, index=range(0,len(author_score.keys())),columns=columns)
df_authors["Author Id"] = author_score.keys()
df_authors["Author Score"] = author_score.values()
df_authors["Number Of Citations"] = df_authors.apply(lambda row: get_author_citation(row["Author Id"]),axis = 1)
df_authors["Number Of Papers"] = df_authors.apply(lambda row: get_author_papers(row["Author Id"]),axis = 1)
df_authors["Author Name"] = df_authors.apply(lambda row: get_author_name(row["Author Id"]),axis = 1)
df_authors["Number Of Publications"] = df_authors.apply(lambda row: get_author_publications(row["Author Id"]),axis = 1)

In [ ]:
del author_list
# del author_score
del cf_score

In [ ]:
# df = tidy_split(df,"Authors","},")

In [ ]:
def get_avg_author_score(row):
    values = row["Authors"].split("},")
    sum_score = 0
    for value in values:
        val = value
        index1 = val.find("AuId")
        index2 = val.find("AfN")
        index3 = val.find("'S'")
        if index3 < index1:
            val = val[index1+4:index2]
            val = val.strip("', :")
        else:
            val = val[index1+4:index3]
            val = val.strip("', :")                
        auth_id = val
        sum_score += author_score[auth_id]
#         sum_score = df_authors[df_authors["Author Id"] == auth_id]["Author Score"]
    return sum_score/row["Number_Of_authors"]

In [ ]:
df["Avg_Author_score"] = df.apply(lambda row: get_avg_author_score(row),axis = 1)

In [ ]:
def get_first_author(row):
    values = row["Authors"].split("},")
    val = values[0]
    index1 = val.find("AuId")
    index2 = val.find("AfN")
    index3 = val.find("'S'")
    if index3 < index1:
        val = val[index1+4:index2]
        val = val.strip("', :")
    else:
        val = val[index1+4:index3]
        val = val.strip("', :")                
    auth_id = val
    return auth_id
def get_first_author_score(row):
    values = row["Authors"].split("},")
    val = values[0]
    index1 = val.find("AuId")
    index2 = val.find("AfN")
    index3 = val.find("'S'")
    if index3 < index1:
        val = val[index1+4:index2]
        val = val.strip("', :")
    else:
        val = val[index1+4:index3]
        val = val.strip("', :")                
    auth_id = val
    return author_score[auth_id]
#     return df[df["Author Id"] == auth_id]["Author Score"]

In [ ]:
df["FirstAuthor"] = df.apply(lambda row: get_first_author(row),axis = 1)
df["FirstAuthorScore"] = df.apply(lambda row: get_first_author_score(row),axis = 1)

In [ ]:
del author_score

In [ ]:
df["Domain_count"] = df.apply(lambda row:get_num_domains(row), axis = 1)

In [ ]:
df.head()

In [ ]:
df.to_csv("final_featured_data.csv")

In [ ]:
features = ['Avg_Author_score','FirstAuthor','FirstAuthorScore','Citation_count','Id','Year','Popularity','Month','Publication_Rank','Publication Type','Number_Of_authors','Year Since Publication','Venue_label','Domain_count','Topic_label']

In [ ]:
df["FirstAuthor"] = df["FirstAuthor"].astype(int)
df["Number_Of_authors"] = df["Number_Of_authors"].astype(int)
df["Publication Type"] = df["Publication Type"].astype(int)
df["Month"] = df["Month"].astype(int)
df["Id"] = df["Id"].astype(int)
df["Year"] = df["Year"].astype(int)
df["Citation_count"] = df["Citation_count"].astype(int)

In [ ]:
df_train = df[df["Year"] <= 2010]
df_dev = df[df["Year"] == 2011]
df_eval = df[df["Year"] > 2011]

In [ ]:
# df.dtypes

In [ ]:
# df.shape

In [ ]:
import lightgbm as lgb 

lgb_params = {"objective" : "regression", "metric" : "rmse",
              "num_leaves" : 70, "learning_rate" : 0.01, 
              "bagging_fraction" : 0.75, "feature_fraction" : 0.8, "bagging_frequency" : 9}
    
lgb_train = lgb.Dataset(df_train[features], label=df_train["page_rank"])
lgb_val = lgb.Dataset(df_dev[features], label=df_dev["page_rank"])
model = lgb.LGBMRegressor(boosting_type='gbdt',num_leaves=31,max_depth=-1, 
learning_rate=0.01, 
n_estimators=1000, 
max_bin=255, 
subsample_for_bin=50000, 
objective=None, 
min_split_gain=0, 
min_child_weight=3,
min_child_samples=10, 
subsample=1, 
subsample_freq=1, 
colsample_bytree=1, 
reg_alpha=0.1, 
reg_lambda=0, 
seed=17,
silent=False, 
nthread=-1)
model.fit(df_train[features], df_train["page_rank"], eval_set=[(df_dev[features], df_dev["page_rank"])])

In [ ]:
pred_dev = model.predict(df_dev[features])
pred_test = model.predict(df_eval[features])

In [ ]:
df_eval['prob'] = pred_test

In [ ]:
import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [ ]:
df_eval['prob'] = df_eval.apply (lambda row: sigmoid(row["page_rank"]),axis=1)
df_eval = df_eval.sort_values(by=['prob'],ascending=False)
print(df_eval[0:100])

In [ ]:
imp_list = model.feature_importances_

In [ ]:
num = sum(imp_list)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp = pd.DataFrame(sorted(zip(model.feature_importances_,features)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-01.png')

In [ ]:
df_eval.to_csv("Model_output_rank.csv")

In [ ]:
df_eval.head()

In [ ]:
author_score = {}
author_list = {}
for index,row in df_eval.iterrows():
    values = row["Authors"].split("},")
    for value in values:
        val = value
        index1 = val.find("AuId")
        index2 = val.find("AfN")
        index3 = val.find("'S'")
        if index3 < index1:
            val = val[index1+4:index2]
            val = val.strip("', :")
        else:
            val = val[index1+4:index3]
            val = val.strip("', :")                
        auth_id = val
        index1 = value.find("AuN")
        val = value[index1+3:]
        val = val.strip("': }]")
        auth_name = val
        auth_info = author_list.get(auth_id,[0,0])
        auth_info[0] += row["page_rank"] * row["prob"]
        auth_info[1] += 1
        author_list[auth_id] = auth_info
for key, value in author_list.items():
    auth_info = author_list.get(key,[0,0])
    author_score[key] = auth_info[0]/auth_info[1]

In [ ]:
def get_author_new_score(auth_id):
    if auth_id in author_score.keys():
        return author_score[auth_id]
    else:
        return 0

In [ ]:
del authr_list
del author_score

In [ ]:
df_authors["Pred Score"] = df_authors.apply(lambda row: get_author_new_score(row["Author Id"]),axis = 1)

In [ ]:
df_authors.to_csv("Author_Table.csv")